In [1]:
import os
os.environ['TORCH_SHOW_CPP_STACKTRACES'] = '0'  # Suppress flash attention warning
os.environ['TORCH_WARN_ONLY_ONCE'] = '1'  # Show warnings only once

directory = r'D:\github\Cricket-Prediction\data\pytorchData'
import pickle
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

class CricketDataset(Dataset):
    def __init__(self, teamStats, playersStats, balltoball):
        self.teamStats = StandardScaler().fit_transform(teamStats)
        self.playersStats = StandardScaler().fit_transform(playersStats.reshape(-1, playersStats.shape[-1])).reshape(playersStats.shape)
        self.balltoball = StandardScaler().fit_transform(balltoball.reshape(-1, balltoball.shape[-1])).reshape(balltoball.shape)

    def __len__(self):
        return len(self.teamStats)
    
    def __getitem__(self, idx):
        team_input = torch.tensor(self.teamStats[idx], dtype=torch.float32)
        player_input = torch.tensor(self.playersStats[idx], dtype=torch.float32)
        ball_stats = torch.tensor(self.balltoball[idx], dtype=torch.float32)
        label = ball_stats[:,-1].mean()
        ball_input = ball_stats[:,:-1]
        return team_input, player_input, ball_input, label
    
# collate_fn to pad the sequences
def collate_fn(batch):
    max_len = 280
    # pad balls upto max_len
    team, player, balls, labels = zip(*batch)
    padded_balls = torch.zeros(len(balls), max_len, balls[0].shape[1])
    for i, ball in enumerate(balls):
        padded_balls[i, :ball.shape[0], :] = ball
    return torch.stack(team), torch.stack(player), padded_balls, torch.tensor(labels, dtype=torch.float32)


train_loader = pickle.load(open(os.path.join(directory, 'train_loader.pkl'), 'rb'))
val_loader = pickle.load(open(os.path.join(directory, 'val_loader.pkl'), 'rb'))
test_loader = pickle.load(open(os.path.join(directory, 'test_loader.pkl'), 'rb'))

# Configuration dictionary
config = {
    'team_model': {
        'input_dim': 23,
        'hidden_dim': 128,
        'output_dim': 64
    },
    'player_model': {
        'input_dim': (1, 22, 22),
        'conv1_out_channels': 64,
        'conv2_out_channels': 64,
        'fc1_input_dim': 64 * 5 * 5,  # Adjust based on the output size of conv layers
        'fc1_output_dim': 128
    },
    'ball_model': {
        'input_dim': 4,
        'transformer': {
            'd_model': 128,
            'nhead': 8,
            'num_encoder_layers': 6,
            'dim_feedforward': 512,
            'dropout': 0.1,
        },
        'output_dim': 128
    },
    'combined_model': {
        'team_model_output_dim': 64,
        'player_model_output_dim': 128,
        'ball_model_output_dim': 128,
        'fc1_output_dim': 128,
        'fc2_output_dim': 1
    }
}

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Team stats model
class TeamStatsModel(nn.Module):
    def __init__(self):
        super(TeamStatsModel, self).__init__()
        self.fc1 = nn.Linear(config['team_model']['input_dim'], config['team_model']['hidden_dim'])
        self.bn1 = nn.BatchNorm1d(config['team_model']['hidden_dim'])
        self.fc2 = nn.Linear(config['team_model']['hidden_dim'], config['team_model']['output_dim'])
        self.bn2 = nn.BatchNorm1d(config['team_model']['output_dim'])
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        return x

# Player stats model
class PlayerStatsModel(nn.Module):
    def __init__(self):
        super(PlayerStatsModel, self).__init__()
        self.conv1 = nn.Conv2d(1, config['player_model']['conv1_out_channels'], kernel_size=(3, 3))
        self.bn1 = nn.BatchNorm2d(config['player_model']['conv1_out_channels'])
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(config['player_model']['conv1_out_channels'], config['player_model']['conv2_out_channels'], kernel_size=(3, 3))
        self.bn2 = nn.BatchNorm2d(config['player_model']['conv2_out_channels'])
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Calculate the size of the flattened layer
        self.flatten_input_dim = self._get_flatten_input_dim(config['player_model']['input_dim'])
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self.flatten_input_dim, config['player_model']['fc1_output_dim'])
        self.bn3 = nn.BatchNorm1d(config['player_model']['fc1_output_dim'])
        self.dropout = nn.Dropout(0.5)

    def _get_flatten_input_dim(self, input_dim):
        x = torch.zeros(1, *input_dim)
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        return x.numel()

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = F.relu(self.bn3(self.fc1(x)))
        x = self.dropout(x)
        return x

# Ball by ball model
class BallByBallModel(nn.Module):
    def __init__(self):
        super(BallByBallModel, self).__init__()
        self.input_projection = nn.Linear(config['ball_model']['input_dim'], config['ball_model']['transformer']['d_model'])
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=config['ball_model']['transformer']['d_model'],
                nhead=config['ball_model']['transformer']['nhead'],
                dim_feedforward=config['ball_model']['transformer']['dim_feedforward'],
                dropout=config['ball_model']['transformer']['dropout'],
                batch_first=True  # Set batch_first to True
            ),
            num_layers=config['ball_model']['transformer']['num_encoder_layers']
        )
        self.fc1 = nn.Linear(config['ball_model']['transformer']['d_model'], config['ball_model']['output_dim'])
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.input_projection(x)
        # No need to permute the dimensions as batch_first=True
        x = self.transformer(x)
        x = x.mean(dim=1)  # Average over sequence_length
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return x

# Combined model
class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.team_model = TeamStatsModel()
        self.player_model = PlayerStatsModel()
        self.ball_model = BallByBallModel()
        self.fc1 = nn.Linear(config['combined_model']['team_model_output_dim'] + config['combined_model']['player_model_output_dim'] + config['combined_model']['ball_model_output_dim'], config['combined_model']['fc1_output_dim'])
        self.bn1 = nn.BatchNorm1d(config['combined_model']['fc1_output_dim'])
        self.fc2 = nn.Linear(config['combined_model']['fc1_output_dim'], config['combined_model']['fc2_output_dim'])
        self.dropout = nn.Dropout(0.5)

    def forward(self, team, player, ball):
        team = team.squeeze(1)  # Remove the extra dimension
        team = self.team_model(team)
        player = self.player_model(player)
        ball = self.ball_model(ball)
        
        # Reshape team tensor to match dimensions
        team = team.view(team.size(0), -1)
        
        # Flatten player tensor to match dimensions
        player = player.view(player.size(0), -1)
        
        x = torch.cat((team, player, ball), dim=1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x


In [ ]:
model = CombinedModel().to(device)  # Ensure the model is moved to the GPU
# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Add weight decay for regularization

def calculate_accuracy(output, labels):
    preds = (output > 0.5).float()
    correct = (preds == labels).float().sum()
    return correct / labels.numel()

epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for i, (team, player, ball, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
        team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)
        
        # Reshape labels to match the model's output shape
        labels = labels.view(-1, 1)
        
        optimizer.zero_grad()
        output = model(team, player, ball)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_accuracy += calculate_accuracy(output, labels).item()
    print(f'Epoch: {epoch+1}, Training Loss: {train_loss/len(train_loader)}, Training Accuracy: {train_accuracy/len(train_loader)}')
    
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for i, (team, player, ball, labels) in enumerate(val_loader):
            team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)
            
            # Reshape labels to match the model's output shape
            labels = labels.view(-1, 1)
            
            output = model(team, player, ball)
            loss = criterion(output, labels)
            val_loss += loss.item()
            val_accuracy += calculate_accuracy(output, labels).item()
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy/len(val_loader)}')

CombinedModel(
  (team_model): TeamStatsModel(
    (fc1): Linear(in_features=23, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
  )
  (player_model): PlayerStatsModel(
    (conv2d): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (maxpool2d): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2d2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=4096, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU()
  )
  (ball_model): BallByBallModel(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        

In [52]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for i, (team, player, ball, labels) in enumerate(train_loader):
        team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(team, player, ball)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for i, (team, player, ball, labels) in enumerate(val_loader):
            team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)
            output = model(team, player, ball)
            loss = criterion(output, labels)
            val_loss += loss.item()
        print(f'Epoch: {epoch}, Validation Loss: {val_loss/len(val_loader)}')


AssertionError: was expecting embedding dimension of 128, but got 4